In [6]:
import tensorflow as tf
import numpy as np

amplifier = 1
threshold_percentage = 50

# Load the pre-trained model
model = tf.keras.models.load_model('MNIST_vgg16_0&1.h5')

# Copy the structure and parameters of the model
freezing_value_mnist = tf.keras.models.clone_model(model)
freezing_value_mnist.set_weights(model.get_weights())

# Modify the weights as specified
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        new_weights = []

        for weight_matrix in weights:
            # Compute the biggest weight magnitude in the whole layer
            max_magnitude = np.max(np.abs(weight_matrix))
            
            # Apply the transformation to each weight
            new_weight_matrix = 1 - (np.abs(weight_matrix) / max_magnitude) ** amplifier
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Further modify the weights by setting the smallest 50% weights for each layer to 0
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        new_weights = []

        for weight_matrix in weights:
            # Flatten the weight matrix and sort the weights by magnitude
            flat_weights = weight_matrix.flatten()
            threshold = np.percentile(np.abs(flat_weights), threshold_percentage)
            
            # Set the smallest 50% weights to 0
            new_weight_matrix = np.where(np.abs(weight_matrix) < threshold, 0, weight_matrix)
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Print all the weights in the model
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        print(f"Weights for layer {layer.name}:")
        for weight_matrix in weights:
            print(weight_matrix)

# Save the modified model
freezing_value_mnist.save('freezing_value_mag_0&1_50%_^1.h5')

print("Model 'freezing_value_mag_0&1_50%_^1.h5' has been saved.")


Weights for layer conv2d:
[[[[0.75747746 0.         0.         ... 0.         0.
    0.99253917]
   [0.6088927  0.         0.         ... 0.         0.
    0.78857666]
   [0.8989447  0.         0.99966794 ... 0.56626886 0.89184767
    0.99701935]]

  [[0.9792113  0.7896273  0.5475897  ... 0.5207259  0.8678351
    0.        ]
   [0.         0.         0.         ... 0.7760576  0.
    0.8907181 ]
   [0.         0.77071035 0.83925813 ... 0.93471247 0.
    0.7562719 ]]

  [[0.62099683 0.9046538  0.         ... 0.         0.
    0.6518841 ]
   [0.         0.         0.         ... 0.         0.
    0.703169  ]
   [0.         0.8104612  0.93235326 ... 0.7884387  0.5909982
    0.        ]]]


 [[[0.90285534 0.69044924 0.         ... 0.61972374 0.
    0.5464788 ]
   [0.         0.9285266  0.         ... 0.9309204  0.
    0.8847773 ]
   [0.         0.         0.87166053 ... 0.6517609  0.8559101
    0.        ]]

  [[0.7021071  0.         0.         ... 0.         0.78687364
    0.89755684]
   [

Weights for layer dense:
[[0.         0.9186381  0.97356325 ... 0.         0.8814423  0.6674911 ]
 [0.90850484 0.92205393 0.         ... 0.81596684 0.90973264 0.82446146]
 [0.6388496  0.7027106  0.8404632  ... 0.9718022  0.6512563  0.68024313]
 ...
 [0.6837096  0.         0.         ... 0.97628593 0.6878892  0.6353704 ]
 [0.         0.8037625  0.83908635 ... 0.         0.         0.        ]
 [0.         0.         0.7318748  ... 0.76310176 0.8785161  0.7878538 ]]
[0.93596226 0.9967749  0.         ... 0.         0.         0.        ]
Weights for layer dense_1:
[[0.7909518  0.         0.80158114 ... 0.8526418  0.         0.        ]
 [0.         0.         0.         ... 0.         0.8374863  0.        ]
 [0.         0.         0.         ... 0.         0.         0.91605747]
 ...
 [0.6196402  0.         0.6237903  ... 0.60915726 0.         0.62139595]
 [0.5939795  0.6296104  0.         ... 0.         0.         0.        ]
 [0.7405529  0.8760209  0.         ... 0.6220953  0.         0